## Omer contour from rembg needs testing

In [1]:
# %pip install rembg
# %pip install onnxruntime

In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [3]:
"""
Tests for the plugin system
"""
import pytest
import os
import sys
import tempfile
from pathlib import Path
import shutil
import importlib
import time
import matplotlib.pyplot as plt
%matplotlib tk


# Import plugin modules
from pyptv import ptv
from pyptv.ptv import py_start_proc_c, py_trackcorr_init, py_sequence_loop



In [ ]:
    
exp_path = Path('/media/user/ExtremePro/omer/exp2')

start = time.time()

try:
    exp_path = Path(exp_path).resolve()
    print(f"Inside main of pyptv_batch, exp_path is {exp_path} \n")
    os.chdir(exp_path)
    
    print(f"double checking that its inside {Path.cwd()} \n")
except Exception:
    raise ValueError(f"Wrong experimental directory {exp_path}")

# RON - make a res dir if it not found

res_path = exp_path / "res"

if not res_path.is_dir():
    print(" 'res' folder not found. creating one")
    res_path.mkdir(parents=True, exist_ok=True)

# read the number of cameras
with open("parameters/ptv.par", "r") as f:
    n_cams = int(f.readline())

cpar, spar, vpar, track_par, tpar, cals, epar = py_start_proc_c(n_cams=n_cams)


first_frame = spar.get_first()
last_frame = spar.get_last()


## For debugging
if last_frame - first_frame > 500:
    last_frame = first_frame + 500

print(f"first frame is {first_frame}")
print(f"last frame is {last_frame}")

# spar.set_first(first_frame)
spar.set_last(last_frame)


exp = {
'cpar':cpar,
'spar':spar,
'vpar':vpar,
'track_par':track_par,
'tpar':tpar,
'cals':cals,
'epar':epar,
'n_cams':n_cams,
    }


# use dataclass to convert dictionary keys to attributes
exp = AttrDict(exp)




Inside main of pyptv_batch, exp_path is /media/user/ExtremePro/omer/exp2 

double checking that its inside /media/user/ExtremePro/omer/exp2 

first frame is 1
last frame is 101


### Next steps: or load plugin from the file or for debugging load the code into this notebook

In [5]:
#    py_sequence_loop(exp)
from pyptv.ptv import run_plugin

# plugin_dir = Path('/home/user/Documents/repos/pyptv/pyptv') / 'plugins'
# sys.path.append(str(plugin_dir))

# print(f"Plugin directory contents: {list(plugin_dir.glob('*.py'))}")

# plugin_file =  plugin_dir / 'ext_sequence_rembg_contour.py'


# if not plugin_file.exists():
#     raise FileNotFoundError(f"Plugin file not found at {plugin_file}")


# plugin = importlib.import_module('ext_sequence_rembg_contour')
# sequence = plugin.Sequence(exp=exp)


In [6]:

# %load /home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_rembg_contour.py
import random

import numpy as np
from imageio.v3 import imread, imwrite
from pathlib import Path

from skimage import img_as_ubyte
from skimage import filters, measure, morphology
from skimage.color import rgb2gray, label2rgb, rgba2rgb
from skimage.segmentation import clear_border
from skimage.morphology import binary_erosion, binary_dilation, disk
from skimage.util import img_as_ubyte

from optv.correspondences import correspondences, MatchedCoords
from optv.tracker import default_naming
from optv.orientation import point_positions

import matplotlib.pyplot as plt

from rembg import remove, new_session
session = new_session('u2net')

def save_mask_areas(areas_data: list, output_file: Path) -> None:
    """Save mask areas to CSV file.
    
    Parameters
    ----------
    areas_data : list
        List of dictionaries containing camera number, frame number, and area
    output_file : Path
        Path to output CSV file
    """
    import pandas as pd
    df = pd.DataFrame(areas_data)
    df.to_csv(output_file, index=False)

def mask_image(imname: Path, display: bool = False) -> tuple[np.ndarray, float]:
    """Mask the image using rembg and keep the entire mask.
    
    Parameters
    ----------
    imname : Path
        Path to the image file
    display : bool
        Whether to display debug plots
        
    Returns
    -------
    tuple[np.ndarray, float]
        Masked image and the area of the mask below row 600 in pixels
    """
    input_data = imread(imname)
    mask = remove(input_data, session=session, only_mask=True)
    
    # Set ROI threshold
    y_threshold = 600
    
    # Create ROI mask below threshold
    roi_mask = np.zeros_like(mask, dtype=bool)
    roi_mask[y_threshold:, :] = True
    
    # Calculate area in ROI
    mask_in_roi = np.where(roi_mask, mask, False)
    area = np.sum(mask_in_roi)
    
    if display:
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Original image
        ax1.imshow(input_data)
        ax1.axhline(y=y_threshold, color='r', linestyle='--')
        ax1.set_title('Original image')
        
        # Full mask
        ax2.imshow(mask)
        ax2.axhline(y=y_threshold, color='r', linestyle='--')
        ax2.set_title('Full mask')
        
        # Masked image
        ax3.imshow(np.where(mask, input_data, 0))
        ax3.axhline(y=y_threshold, color='r', linestyle='--')
        ax3.set_title('Masked image')
        
        # ROI masked image
        ax4.imshow(np.where(mask_in_roi, input_data, 0))
        ax4.set_title(f'ROI mask (area: {area} pixels)')
        
        plt.tight_layout()
        plt.show()

    # Apply the mask to the input image
    masked_image = np.where(mask, input_data, 0)
    return masked_image, area

class Sequence:
    """Sequence class defines external tracking addon for pyptv
    User needs to implement the following functions:
            do_sequence(self)

    Connection to C ptv module is given via self.ptv and provided by pyptv software
    Connection to active parameters is given via self.exp1 and provided by pyptv software.

    User responsibility is to read necessary files, make the calculations and write the files back.
    """

    def __init__(self, ptv=None, exp=None):
        self.ptv = ptv
        self.exp = exp
        self.areas_data = []  # Store areas data during processing

    def do_sequence(self):
        """ Copy of the sequence loop with one change we call everything as 
        self.ptv instead of ptv. 
        
        """
        # Sequence parameters    

        n_cams, cpar, spar, vpar, tpar, cals = (
            self.exp.n_cams,
            self.exp.cpar,
            self.exp.spar,
            self.exp.vpar,
            self.exp.tpar,
            self.exp.cals,
        )

        # # Sequence parameters
        # spar = SequenceParams(num_cams=n_cams)
        # spar.read_sequence_par(b"parameters/sequence.par", n_cams)


        # sequence loop for all frames
        first_frame = spar.get_first()
        last_frame = spar.get_last()
        print(f" From {first_frame = } to {last_frame = }")
        
        for frame in range(first_frame, last_frame + 1):
            # print(f"processing {frame = }")

            detections = []
            corrected = []
            for i_cam in range(n_cams):
                base_image_name = spar.get_img_base_name(i_cam)
                imname = Path(base_image_name % frame) # works with jumps from 1 to 10 
                masked_image, area = mask_image(imname, display=False)

                # Store area data
                self.areas_data.append({
                    'camera': i_cam,
                    'frame': frame,
                    'area': area
                })                

                # img = imread(imname)
                # if img.ndim > 2:
                #     img = rgb2gray(img)
                    
                # if img.dtype != np.uint8:
                #     img = img_as_ubyte(img)

                        
                
                high_pass = self.ptv.simple_highpass(masked_image, cpar)
                targs = self.ptv.target_recognition(high_pass, tpar, i_cam, cpar)

                targs.sort_y()
                detections.append(targs)
                masked_coords = MatchedCoords(targs, cpar, cals[i_cam])
                pos, _ = masked_coords.as_arrays()
                corrected.append(masked_coords)

            #        if any([len(det) == 0 for det in detections]):
            #            return False

            # Corresp. + positions.
            sorted_pos, sorted_corresp, _ = correspondences(
                detections, corrected, cals, vpar, cpar)

            # Save targets only after they've been modified:
            # this is a workaround of the proper way to construct _targets name
            for i_cam in range(n_cams):
                base_name = spar.get_img_base_name(i_cam)
                # base_name = replace_format_specifiers(base_name) # %d to %04d
                self.ptv.write_targets(detections[i_cam], base_name, frame)

            print("Frame " + str(frame) + " had " +
                repr([s.shape[1] for s in sorted_pos]) + " correspondences.")

            # Distinction between quad/trip irrelevant here.
            sorted_pos = np.concatenate(sorted_pos, axis=1)
            sorted_corresp = np.concatenate(sorted_corresp, axis=1)

            flat = np.array([
                corrected[i].get_by_pnrs(sorted_corresp[i])
                for i in range(len(cals))
            ])
            pos, _ = point_positions(flat.transpose(1, 0, 2), cpar, cals, vpar)

            # if len(cals) == 1: # single camera case
            #     sorted_corresp = np.tile(sorted_corresp,(4,1))
            #     sorted_corresp[1:,:] = -1

            if len(cals) < 4:
                print_corresp = -1 * np.ones((4, sorted_corresp.shape[1]))
                print_corresp[:len(cals), :] = sorted_corresp
            else:
                print_corresp = sorted_corresp

            # Save rt_is
            rt_is_filename = default_naming["corres"].decode()
            rt_is_filename = rt_is_filename + f'.{frame}'
            with open(rt_is_filename, "w", encoding="utf8") as rt_is:
                rt_is.write(str(pos.shape[0]) + "\n")
                for pix, pt in enumerate(pos):
                    pt_args = (pix + 1, ) + tuple(pt) + tuple(print_corresp[:, pix])
                    rt_is.write("%4d %9.3f %9.3f %9.3f %4d %4d %4d %4d\n" % pt_args)

            
       
        # After processing all frames, save the areas data
        output_file = Path('res/mask_areas.csv')
        save_mask_areas(self.areas_data, output_file)
        print(f"Mask areas saved to {output_file}")


### Here's the importan bit

In [7]:
sequence = Sequence(ptv = ptv, exp=exp)
sequence.do_sequence()


 From first_frame = 1 to last_frame = 101
Frame 1 had [89, 313, 164] correspondences.
Frame 2 had [87, 335, 158] correspondences.
Frame 3 had [88, 345, 152] correspondences.
Frame 4 had [85, 330, 160] correspondences.
Frame 5 had [76, 343, 152] correspondences.
Frame 6 had [79, 339, 157] correspondences.
Frame 7 had [87, 340, 143] correspondences.
Frame 8 had [87, 340, 151] correspondences.
Frame 9 had [91, 323, 150] correspondences.
Frame 10 had [88, 333, 137] correspondences.
Frame 11 had [83, 343, 138] correspondences.
Frame 12 had [86, 327, 138] correspondences.
Frame 13 had [85, 336, 132] correspondences.
Frame 14 had [86, 323, 131] correspondences.
Frame 15 had [92, 317, 141] correspondences.
Frame 16 had [96, 320, 135] correspondences.
Frame 17 had [97, 322, 131] correspondences.
Frame 18 had [96, 314, 136] correspondences.
Frame 19 had [93, 319, 129] correspondences.
Frame 20 had [97, 311, 141] correspondences.
Frame 21 had [95, 327, 132] correspondences.
Frame 22 had [99, 316,

In [8]:

tracker = py_trackcorr_init(exp)
tracker.full_forward()

end = time.time()
print("time lapsed %f sec" % (end - start))



 Renaming ../Runs_1003/Cam0_2025-03-10-13.32.35/%08d.tif to ../Runs_1003/Cam0_2025-03-10-13.32.35/ before C library tracker
 Renaming ../Runs_1003/Cam1_2025-03-10-13.33.28/%08d.tif to ../Runs_1003/Cam1_2025-03-10-13.33.28/ before C library tracker
 Renaming ../Runs_1003/Cam2_2025-03-10-13.34.40/%08d.tif to ../Runs_1003/Cam2_2025-03-10-13.34.40/ before C library tracker
 Renaming ../Runs_1003/Cam3_2025-03-10-13.36.08/%08d.tif to ../Runs_1003/Cam3_2025-03-10-13.36.08/ before C library tracker
step: 1, curr: 566, next: 580, links: 243, lost: 323, add: 7
time lapsed 328.945898 sec
step: 2, curr: 580, next: 592, links: 293, lost: 287, add: 10
step: 3, curr: 592, next: 585, links: 293, lost: 299, add: 7
step: 4, curr: 585, next: 578, links: 295, lost: 290, add: 6
step: 5, curr: 578, next: 581, links: 294, lost: 284, add: 12
step: 6, curr: 581, next: 582, links: 285, lost: 296, add: 8
step: 7, curr: 582, next: 586, links: 279, lost: 303, add: 6
step: 8, curr: 586, next: 571, links: 264, lost:

In [ ]:
def analyze_mask_areas(csv_file='res/mask_areas.csv'):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), height_ratios=[2, 1])
    
    # Plot 1: Area over time
    for cam in sorted(df['camera'].unique()):
        cam_data = df[df['camera'] == cam]
        ax1.plot(cam_data['frame'], cam_data['area'], 
                label=f'Camera {cam}', 
                marker='o', 
                markersize=4, 
                alpha=0.7)
    
    ax1.set_xlabel('Frame Number')
    ax1.set_ylabel('Mask Area (pixels)')
    ax1.set_title('Mask Areas Over Time by Camera')
    ax1.legend()
    ax1.grid(True)
    
    # Plot 2: Box plot of areas by camera
    sns.boxplot(data=df, x='camera', y='area', ax=ax2)
    ax2.set_title('Distribution of Mask Areas by Camera')
    ax2.set_xlabel('Camera')
    ax2.set_ylabel('Area (pixels)')
    
    # Print statistics
    print("\nMask Area Statistics:")
    print("-" * 40)
    stats = df.groupby('camera')['area'].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max')
    ]).round(2)
    print(stats)
    
    # Adjust layout
    plt.tight_layout()
    
    # Save the plot
    plt.savefig('res/mask_areas_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:

# analyze_mask_areas()


Mask Area Statistics:
----------------------------------------
                mean        std        min        max
camera                                               
0       1.937634e+08  434109.37  192819778  194472577
1       1.717797e+08  955954.05  169572324  172860594
2       1.592121e+08  887826.20  157253801  160304740
3       1.475163e+08  799199.88  145694511  148504378


In [12]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_normalized_areas(csv_file='res/mask_areas.csv'):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Create simple plot
    plt.figure(figsize=(10, 6))
    
    # Plot one line per camera, normalized by first value
    for cam in sorted(df['camera'].unique()):
        cam_data = df[df['camera'] == cam]
        # Normalize by the first value
        normalized_area = cam_data['area'] / cam_data['area'].iloc[0]
        plt.plot(cam_data['frame'], normalized_area, label=f'Camera {cam}')
    
    plt.xlabel('Frame')
    plt.ylabel('Normalized Area')
    plt.legend()
    plt.grid(True)
    plt.savefig('res/normalized_areas.png')
    plt.show()

# if __name__ == "__main__":
plot_normalized_areas()